<a href="https://colab.research.google.com/github/wadaka0821/nlp-tutorial/blob/main/answers/7_3_MLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# BERT の利用(MLM)
## 作成者：和田
## 最終更新日：2023/03/26

In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 53.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 35.9 MB/s eta 0:00:00


## Hugging Face の transformers ライブラリを使用して，事前学習済みの BERT を使用してみます

In [2]:
from transformers import AutoTokenizer, AutoModelForMaskedLM
import torch

In [3]:
# トークナイザの読み込み
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# BERTの読み込み
model = AutoModelForMaskedLM.from_pretrained("bert-base-uncased")
# 今回は学習しないので推論モードに切り替え
model.eval()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tr

**Hi! Nice to meet you! My name is John Smith. I am 19 and a student in college.**
の 「college」をマスクして，予測してみます

In [4]:
text = "Hi! Nice to meet you! My name is John Smith. I am 19 and a student in [MASK]."

## STEP1 : 文(str)からトークンへの変換

In [5]:
# トークンを得たいだけの時は return_tensors を指定する必要はありません
# 入力が複数ある場合(入力の型が List[str]のとき)は padding, truncation, max_length等の引数を必要に応じて指定する必要があります．
tokenized_text = tokenizer(text, return_tensors='pt')

In [6]:
# トークナイズされた文を確認
# input_ids は トークンの id
# token_type_ids は セグメントの情報
# attention_mask は padding 等をしたときのマスクの情報
# をそれぞれ表します
tokenized_text

{'input_ids': tensor([[ 101, 7632,  999, 3835, 2000, 3113, 2017,  999, 2026, 2171, 2003, 2198,
         3044, 1012, 1045, 2572, 2539, 1998, 1037, 3076, 1999,  103, 1012,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [7]:
# id -> str の逆の変換をしてみる
# BERT では最初に [CLS], 素性の区切り，入力の終わりには [SEP] を標準で挿入します．
# 自分で特殊トークンを追加する方法もあります．
# その際は tokenizer へのトークンの追加だけではなく BERT の Embedding の vocab_size の変更を忘れないように！
# また，使用しているモデルが uncased なので全て小文字になっています．
tokenizer.decode(tokenized_text.input_ids[0])

'[CLS] hi! nice to meet you! my name is john smith. i am 19 and a student in [MASK]. [SEP]'

## STEP2 : BERT に入力する

In [8]:
# トークナイズしたときに得た input_ids, token_type_ids, attention_mask はキーワード引数としてモデルに与えます
logits = model(**tokenized_text).logits

## STEP3 : マスクの部分の予測結果を得る

In [9]:
# [MASK] の部分だけ予測結果から最も確率の高いトークンを選択します
filled_text_ids = list()
for i in range(len(tokenized_text.input_ids[0])):
    if tokenized_text.input_ids[0, i].item() == tokenizer.mask_token_id:
        filled_text_ids.append(torch.argmax(logits[0, i, :]).item())
    else:
        filled_text_ids.append(tokenized_text.input_ids[0, i].item())

## STEP4 : デコードして結果を確認

In [10]:
# 結果を確認
tokenizer.decode(filled_text_ids)

'[CLS] hi! nice to meet you! my name is john smith. i am 19 and a student in college. [SEP]'

# 問題1
---
上で行った処理をバッチで実行する関数を書いてください．  
↓例


```python
# 入力
texts = ["Hi! Nice to meet you! My name is John Smith. I am 19 and a student in [MASK].",
         "I have a younger [MASK]. He just started high school. He is [MASK] and lives with my parents."]

fillted_texts = batch_fill_mask(texts)

print(fillted_texts)

# ["Hi! Nice to meet you! My name is John Smith. I am 19 and a student in college.",
#  "I have a younger brother. He just started high school. He is 15 and lives with my parents."]
```



## 解答例

In [12]:
texts = ["Hi! Nice to meet you! My name is John Smith. I am 19 and a student in [MASK].",
         "I have a younger [MASK]. He just started high school. He is [MASK] and lives with my parents."]

In [33]:
def batch_fill_mask(texts, model, tokeizer):
    tokenized_texts = tokenizer(texts, return_tensors='pt', padding=True, truncation=True, max_length=512)
    with torch.no_grad():
        logits = model(**tokenized_texts).logits
        pred_ids = torch.argmax(logits, dim=2)
        mask_tokens_filter = (tokenized_texts.input_ids == tokenizer.mask_token_id).long()
        pred_tokens = pred_ids * mask_tokens_filter
        pred_tokens = pred_tokens + tokenized_texts.input_ids * (torch.ones(*mask_tokens_filter.shape).long() - mask_tokens_filter)
    decoded_texts = tokenizer.batch_decode(pred_tokens, skip_special_tokens=True)
    return decoded_texts

In [34]:
batch_fill_mask(texts, model, tokenizer)

['hi! nice to meet you! my name is john smith. i am 19 and a student in college.',
 'i have a younger brother. he just started high school. he is married and lives with my parents.']

問題2
---
例で行った推論方法ではマスクされた部分のトークンの出力で最も高い確率のトークンを推論結果としていました．  
確率上位 k 件からランダムにトークンを予測するような関数を書いてください．

## 解答例

In [35]:
text = "Hi! Nice to meet you! My name is John Smith. I am 19 and a student in [MASK]."

In [56]:
def fill_mask_topk(text, topk, model, tokenizer):
    tokenized_text = tokenizer(text, return_tensors='pt')
    out_ids = list()
    with torch.no_grad():
        logits = model(**tokenized_text).logits
        pred_ids = torch.topk(logits[0], k=topk, dim=1).indices
        for i in range(len(tokenized_text.input_ids[0])):
            if tokenized_text.input_ids[0, i].item() == tokenizer.mask_token_id:
                perm = torch.randperm(topk)
                out_ids.append(pred_ids[i, perm][0].item())
            else:
                out_ids.append(tokenized_text.input_ids[0, i].item())
    return tokenizer.decode(out_ids, skip_special_tokens=True)

In [57]:
fill_mask_topk(text, 5, model, tokenizer)

'hi! nice to meet you! my name is john smith. i am 19 and a student in engineering.'

In [58]:
fill_mask_topk(text, 5, model, tokenizer)

'hi! nice to meet you! my name is john smith. i am 19 and a student in psychology.'

In [59]:
fill_mask_topk(text, 5, model, tokenizer)

'hi! nice to meet you! my name is john smith. i am 19 and a student in college.'

In [60]:
fill_mask_topk(text, 5, model, tokenizer)

'hi! nice to meet you! my name is john smith. i am 19 and a student in physics.'

In [61]:
fill_mask_topk(text, 5, model, tokenizer)

'hi! nice to meet you! my name is john smith. i am 19 and a student in school.'